In [1]:
# set conda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

# install psi4
!conda install -y psi4 python=3.7 -c psi4

# set path
import sys
sys.path.append("/usr/local/lib/python3.7/site-packages/")

# this command is needed to avoid "Loader" error.
!pip install distributed

--2022-04-26 02:28:28--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2022-04-26 02:28:28--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75660608 (72M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  72.16M  89.3MB/s    in 0.8s    

2022-04-26 02:28:29 (89.3 MB/s) - ‘Miniconda3-latest-Linu

In [2]:
# import Psi4
import psi4
# check Psi4 version
print(psi4.__version__)

1.5


In [32]:
psi4.set_output_file("log.txt")

psi4.set_memory('10GB')
psi4.set_num_threads(2)

mol = psi4.geometry("""
symmetry c1
0 1
O       -1.0239104333      1.3152089264     -0.0149071027                 
H       -0.0345840622      1.2807129464      0.0105245830                 
H       -1.3132760473      0.5529382075      0.5470664770                 
""")

level = 'B3LYP/6-31G(d)'
#level = 'B3LYP/6-31G(d,p)' # Cation
#level = 'B3LYP/6-31+G(d)' # Anions and excited states
#level = 'B3LYP/6-31++G(d)' # Hydride
#level = 'B3LYP/6-311G(d)' # Fluorine
#level = 'MP2/6-31+G(d,p)'
#level = 'wb97x-d/cc-pvtz'
#level = 'M06-D3/6-311G(d,p)' # metal
#level = 'M06-2X-D3/6-311G(d,p)' # non-metal
#level = 'B97-D3BJ/6-311G(d,p)'
#level = 'B3LYP-D3BJ/6-311G(d,p)'
#level = 'CAM-B3LYP/cc-pvtz'
#level = 'CAM-B3LYP-D3BJ/cc-pvtz'
#level = 'HF/3-21G' # test calculation

scf_e, wfn = psi4.optimize(level,return_wfn = True)

Optimizer: Optimization complete!


In [33]:
!cat log.txt


  Memory set to   9.313 GiB by Python driver.
  Threads set to 2 by Python driver.

Scratch directory: /tmp/

Scratch directory: /tmp/
gradient() will perform analytic gradient computation.

*** tstart() called on a1563b19f52b
*** at Tue Apr 26 03:54:12 2022

   => Loading Basis Set <=

    Name: 6-31G(D)
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   145 file /usr/local/share/psi4/basis/6-31g_d_.gbs 
    atoms 2-3 entry H          line    44 file /usr/local/share/psi4/basis/6-31g_d_.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RKS Reference
                        2 Threads,   9536 MiB Core
         ---------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in An

In [5]:
xyz = mol.save_string_xyz()
print (xyz)

0 1
 O    0.000000000004   -0.000000000000    0.067013128142
 H    0.761513003012   -0.000000000000   -0.531773486862
 H   -0.761513003083    0.000000000000   -0.531773486812



In [6]:
fchk_writer = psi4.core.FCHKWriter(wfn)
fchk_writer.write("case.fchk")

In [34]:
scf_e, wfn = psi4.frequency(level, return_wfn = True, ref_gradient=wfn.gradient())

 7 displacements needed.
 1 2 3 4 5 6 7


In [8]:
!pip install py3Dmol

In [12]:
import py3Dmol
print(py3Dmol.__version__)

1.8.0


In [31]:
!cat log.txt

cat: log.txt: No such file or directory


In [188]:
ld = open('log.txt')
lines = ld.readlines()

read_line = 0
cline = ''
for line in lines:
  if read_line > 0:
    if line.strip() == '':
        read_line = 0
        cline += "\n"
    cline += line
  if line.find("Vibration             ") >= 0:
    nv = line.split()
    cline += line
    read_line = 1
print (cline)

  Vibration                       7                   8                   9           
  Freq [cm^-1]                1713.0407           3727.0369           3849.0033       
  Irrep                           A                   A                   A           
  Reduced mass [u]              1.0825              1.0453              1.0810        
  Force const [mDyne/A]         1.8716              8.5553              9.4357        
  Turning point v=0 [a0]        0.2548              0.1758              0.1701        
  RMS dev v=0 [a0 u^1/2]        0.1875              0.1271              0.1251        
  Char temp [K]               2464.6842           5362.3763           5537.8588       
  ----------------------------------------------------------------------------------
      1   O               -0.00 -0.00 -0.07   -0.00 -0.00  0.05   -0.07  0.00 -0.00   
      2   H                0.43 -0.00  0.56    0.58 -0.00 -0.40    0.55 -0.00 -0.44   
      3   H               -0.43 -0.00  0.56  

In [189]:
#dx, dy, and dz data
#  Vibration                       7                   8                   9           
#  Freq [cm^-1]                1713.0407           3727.0379           3849.0033       
#  Irrep                           A                   A                   A           
#  Reduced mass [u]              1.0825              1.0453              1.0810        
#  Force const [mDyne/A]         1.8716              8.5553              9.4357        
#  Turning point v=0 [a0]        0.2548              0.1758              0.1701        
#  RMS dev v=0 [a0 u^1/2]        0.1875              0.1271              0.1251        
#  Char temp [K]               2464.6842           5362.3777           5537.8588       
#  ----------------------------------------------------------------------------------
#      1   O               -0.00  0.00 -0.07    0.00  0.00  0.05   -0.07  0.00  0.00   
#      2   H                0.43  0.00  0.56    0.58 -0.00 -0.40    0.55  0.00 -0.44   
#      3   H               -0.43 -0.00  0.56   -0.58 -0.00 -0.40    0.55 -0.00  0.44  

In [203]:
# set vibration number
vibration_No = 9

In [204]:
for i in range(len(nv)):
  if nv[i] == str(vibration_No):
    read_no = i
#print(read_no)

ld = open('log.txt')
lines = ld.readlines()
out_line = '\n\n'
read_line = 0
for line in lines:
  if read_line > 0:
    read_line += 1
    if line.strip() == '':
        read_line = 0
    if read_line > 9:
      dx = line.split()[(read_no-1)*3+2]
      dy = line.split()[(read_no-1)*3+3]
      dz = line.split()[(read_no-1)*3+4]
      #print(dx,dy,dz)
      out_line += ' '+str(dx)+' '+str(dy)+' '+str(dz)+'\n'
  if line.find("Vibration             ") >= 0:
    read_line = 1

with open('py3Dmol_dr.xyz', 'w') as f:
  print(out_line, file=f)

!cat py3Dmol_dr.xyz



 -0.07 0.00 -0.00
 0.55 -0.00 -0.44
 0.55 -0.00 0.44



In [205]:
natom = xyz.count('\n')-1
py3Dmol_data = xyz.replace('0 1', str(natom)+'\n comment line')

with open('py3Dmol_xyz.xyz', 'w') as f:
  print(py3Dmol_data, file=f)

!cat py3Dmol_xyz.xyz

3
 comment line
 O    0.000000000004   -0.000000000000    0.067013128142
 H    0.761513003012   -0.000000000000   -0.531773486862
 H   -0.761513003083    0.000000000000   -0.531773486812



In [206]:
#!paste py3Dmol_xyz.xyz py3Dmol_dr.xyz > py3Dmol.xyz
#!cat py3Dmol.xyz

def readlines_file(file_name):
    with open(file_name, 'r') as file:
        return file.readlines()

def save_file(file_name, text):
    with open(file_name, 'w') as file:
        file.write(text)

cal1 = readlines_file('py3Dmol_xyz.xyz')
cal2 = readlines_file('py3Dmol_dr.xyz')

cal1 = list(map(lambda x: x.strip(), cal1))
cal2 = list(map(lambda x: x.strip(), cal2))

lines = ["{0} {1}\n".format(line1, line2) for line1, line2 in zip(cal1, cal2)]

save_file('py3Dmol.xyz', "".join(lines))

!cat py3Dmol.xyz

3 
comment line 
O    0.000000000004   -0.000000000000    0.067013128142 -0.07 0.00 -0.00
H    0.761513003012   -0.000000000000   -0.531773486862 0.55 -0.00 -0.44
H   -0.761513003083    0.000000000000   -0.531773486812 0.55 -0.00 0.44
 


In [207]:
##-----py3Dmol.xyz-----
#number of atoms 
#comment
#element1 x y z dx dy dz
#element2 x y z dx dy dz
#...
#elementN x y z dx dy dz
##-----

In [208]:
import py3Dmol

with open('py3Dmol.xyz', 'r') as f:
    file = f.read()

view = py3Dmol.view(width=680, height=480)
view.addModel(file, 'xyz', {'vibrate': {'frames': 10, 'amplitude':1}})
view.setStyle({'stick': {}})
view.animate({'loop': 'backAndForth'})
view.setBackgroundColor('#e1e1e1')
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol